#### Física de la Radioterapia. Master de Física Biomédica. Universidad Complutense de Madrid
# Calibración del acelerador lineal
## Haces de fotones medidos en condiciones de referencia
-----

**Objetivos**:
Determinar la dosis absorbida producida por un campo de radiación en condiciones estándar de calibración utilizando una cámara de ionización tipo Farmer y aplicando el [protocolo de dosimetría de la IAEA TRS398](https://www-pub.iaea.org/MTCD/publications/PDF/TRS_398s_Web.pdf). Se recomienda leer al menos el epígrafe **3. FORMALISMO BASADO EN** $N_{D,w}$ del protocolo.

**Antecendentes**: De acuerdo al TRS398 la magnitud $D_{w, Q}$, dosis en agua para la calidad de la radiación $Q$, se puede determinar mediante la expresión

\begin{equation}
    D_{w, Q} = M_Q\;N_{D, w, Q_0}\;k_{Q, Q_0}\;k_{sat}\;k_{pol}.
\end{equation}

donde:
- $M_Q$ es la lectura neta del electrómetro expresado en condiciones normales
- $N_{D, w, Q_0}$ es el valor del factor de calibración de la cámara en condiciones de referencia
- $k_{Q, Q_0}$ es el factor de corrección por la calidad del haz del factor de calibración
- $k_{sat}$ es el factor de corrección por saturación de la cámara
- $k_{pol}$ es factor de corrección por dependencia de la lectura de la cámara con la polarización de sus electrodos

**Datos**:
Los datos de la cámara empleada están recogidos en su [certificado de calibración](https://raw.githubusercontent.com/csarux/TutorialesFisicaRadioterapia/main/Enunciados/CalibracionAcelerador/docs/CamaraFarmer10820.pdf)

Previamente se ha determinado que para esta cámara los factores de corrección por saturación $k_{sat}$ y por polarización $k_{pol}$ son ambos igual a uno en las condiciones de medida consideradas.

La energía del haz empleado corresponde a una razón TPR${}_{20, 10} = 0.694$.

Para recolectar la carga producida en la cámara se utiliza un electrómetro con una resolución de $0.001$ nC.

## Cuestión
1. **Obtener de los datos del TRS398 el factor de corrección de la calibración de la cámara por la energía para el haz utilizado.**

**Ayuda:** Consultar el CUADRO 14 página 85.

La calidad del haz es TPR=0.694. Para obtener un ajuste bastante adecuado le damos a TPR unos valores entre 0.68 y 0.70.

También calculamos el error utilizando la incertidumbre que se da en el certificado de calibración (1,1%):

In [1]:
a = (0.68 - 0.70)
b = (0.990 - 0.988)
TPR = 0.694

k_694 = b/a * (TPR - 0.7) + 0.988

error_TPR = TPR * 0.011
k_694_errormas = b/a * (TPR + error_TPR - 0.7) + 0.988
k_694_errormenos = b/a * (TPR - error_TPR - 0.7) + 0.988
error_k_694 = abs(k_694_errormas - k_694_errormenos) / 2

print("k_694 =", k_694)
print("error_k_694 =", error_k_694)


k_694 = 0.9886
error_k_694 = 0.0007634000000000252


Por lo tanto el factor de corrección de calibración de la cámara por la energía para el haz es: $k_{Q,Q_0}=0.9886 \pm 0.0008$

## Módulo `simcaliblinac`
Este cuaderno simula las condiciones variables de la medida mediante generadores aleatorios que siguen distribuciones realistas. Para realizar el ejercicio cada alumno generará y recogerá tanto las medidas como las condiciones en las que las ha realizado.

El módulo `simcaliblinac` contiene las funciones para realizar esta simulación.

```
Nota: El código de este modulo se considera aún en desarrollo.
Si al instalar el código aparecen mensajes de error o no es posible localizarlo escribir a cesaro02@ucm.es  
```

En la siguiente celda instalamos el paquete e importamos el módulo

In [2]:
%pip install --index-url https://test.pypi.org/simple/ simcaliblinac
import simcaliblinac.simcaliblinac as scl

Looking in indexes: https://test.pypi.org/simple/


Tomar los valores de la presión atmosférica indicada por el barómetro del Servicio y de la temperatura medida por la sonda del termómetro que introducimos en el agua.

Las unidades de las medidas son presión en hPa y temperatura en grados centígrados.

In [3]:
presion, temperatura = scl.presion_temperatura_medida()
print('Presión=', presion, ' hPa')
print('Temperatura=',temperatura, 'ºC')

Presión= [958.1]  hPa
Temperatura= [22.9] ºC


El módulo `simcaliblinac` ofrece una función para calcular la corrección por presión y temperatura $\phi$pT

In [4]:
ϕpT = scl.ϕpTf(presion, temperatura)
print('Corr por presión y temperatura=', φpT)

Corr por presión y temperatura= [1.06775674]


Una vez montados los equipos, fijadas las condiciones experimentales y con todas las conexiones realizadas, comenzamos por medir la lecutra del electrómetro cuando no hay radiación. En Radioterapia a este valor lo denominamos *corriente de fugas*. Es el equivalente a una lectura de un valor de fondo pero de origen diferente.

Las fugas son de naturaleza aleatoria. Su valor medio se estima integrando durante un tiempo dado suficientemente largo como para promediar variaciones instantáneas.

El módulo `simcaliblinac` expone un método para obtener una muestra de las fugas recolectadas durante un tiempo dado

In [5]:
fugas = scl.fugas(minutos=2)
print('Fugas=', fugas, 'nC')

Fugas= [[0.17]] nC


Para hacer las medidas en el acelerador tenemos que fijar un número de unidades monitor nominales.

In [6]:
UM_nominales = 200

El módulo `simcaliblinac` expone un método para obtener una muestra de **n** medidas con las unidades monitor fijadas. La tasa de UM fijada en el acelerador es de 600 UM/min. Usualmente se toma una muestra de tamaño tres, porque es la menor muestra que permite tomar una decisión sobre la coherencia de las medidas. Tres medidas permiten asegurar que no ha ocurrido nada anómalo al tomar los datos si ningún valor discrepa mucho de los otros dos.

In [7]:
lecturas = scl.lectura_medida_f(UM=UM_nominales, n=3)
print('Lecturas=', lecturas, 'nC')

Lecturas= [[35.71  35.719 35.722]] nC


Para calibrar el acelerador expresamos la medida de dosis en términos relativos a las unidades monitor, es decir, queremos saber cuánta dosis damos por unidad monitor. Por una decisión complemtamente arbitraria se suele ajustar la calibración para que en condiciones estándar una UM coincida con un centiGray. Este ajuste se realiza variando la ganancia de la cámaras monitoras. El cambio de la ganancia permite variar la lectura de la cámara aún recibiendo la misma dosis.

## Cuestiones
2. Determinar la dosis absorbida suministrada por el acelerador y el output.
1. Hacer una estimación de la incertidumbre en la medida de la dosis absorbida y en la medida del output

Con respecto a la **cuestión 2**, para calcular la dosis absorbida suministrada por el acelerador primero debemos calcular la dosis suministrada por el acelerador, que se calcula con la siguiente expresión:
$D_{w,Q}=M_QN_{D,w,Q_0}k_{Q,Q_0}k_{sat}k_{pol}$

Donde en ella, $M_Q$ es la lectura neta del electrómetro en condiciones normales, y su expresión es:
$M_Q=Φ_{pT}(M_{lectura}-M_{fugas})$

En la parte de contribución de fugas se tiene en cuenta la tasa con la que se han realizado las medidas y el tiempo que ha llevado tomarlas.
Sabiendo que el número de unidades monitor nominales en el acelerador es 200 UM, que el tiempo de las fugas recolectadas es de 2 minutos y que la tasa de unidades monitor en el acelerador son 600 UM/minuto. Así, corregimos las fugas con la siguiente expresión:
$$M_{fugas}(200/600 min) = M_{fugas}(2 min)\frac{200/600 min}{2 min}$$

Donde $N_{D,w,Q_0}=(5.396 \pm 0.001)·10^7$ Gy/C es el factor de calibración de la cámara en condiciones de referencia (ya nos viene especificada).

En el que $k_{Q,Q_0}$ es el factor de corrección por la calidad del haz del factor de calibración: $k_{Q,Q_0}=0.989 \pm 0.001$

Y donde $k_{sat}$ y $k_{pol}$ son los factores de corrección por saturación de la cámara y de corrección por dependencia de la lectura de la cámara con la polarización de sus electrodos respectivamente. Sus valores son:
$k_{sat}=1.000 \pm 0001$ y $k_{pol}=1$. Este último adquiere ese valor ya que no se tiene en cuenta por el bajo efecto de la polarización.

Conociendo ya todos estos valores calculamos la dosis absorbida por el acelerador:


In [8]:
import numpy as np
fugas = fugas*(200/600)/2
M_Q = φpT*(lecturas-fugas)*(10**-9)
N_DwQ0 = 5.396e7
k_QQ0 = k_694
k_sat = 1
k_pol = 1

D_wQ = M_Q*N_DwQ0*k_QQ0*k_sat*k_pol
print('Dosis absorbida por el acelerador:', np.mean(D_wQ), 'Gy')

Dosis absorbida por el acelerador: 2.0328025248865607 Gy


Y ahora calculamos la **dosis absorbida suministrada por el output**. Para ello hay que dividir la lectura neta del electrómetro en condiciones normales entre las unidades nominales:

In [9]:
D_UM=D_wQ/UM_nominales*100
print('Dosis absorbida por el output:', np.mean(D_UM), 'cGy')

Dosis absorbida por el output: 1.0164012624432803 cGy


Y ahora respecto a la **cuestión 3** en la que se nos pide hacer una estimación de la incertidumbre en la medida de la dosis absorbida y en la medida del output, se deben valorar los errores estadísticos y los errores de cada una de las variables que aparecen.

Trataremos a continuación el caso de las variables, es decir, los errores que aparecen debido a ellas:

Para ello, comenzamos con el error estadístico en el electrómetro, que aparece a partir de tres medidas tomadas, y se considera aleatorio:

$$
\Delta M_{ale} = \sqrt{\frac{\sum\limits_{i=1}^{n} (M_i - \overline{M})^2}{n(n-1)}}
$$

Donde $M_i$ es cada medida tomada, $\overline{M}$ es la media de esas medidas, y n es el número de medidas tomadas.

In [10]:
DeltaMale=0
for i in range(len(lecturas[0])):
  DeltaMale+=(lecturas[0][i]-np.mean(lecturas[0]))**2/(len(lecturas[0])*(len(lecturas[0])-1))
print(DeltaMale)

1.3000000000000985e-05


Ahora, teniendo en cuenta que también hay un error sistemático que vale $ΔM_{sist}=0.001$ nC, calculamos el error total de lectura del electrómetro:
$$
\Delta M_{total} = \sqrt{\Delta M_{ale}^{2} + \Delta M_{sist}^{2}}
$$


In [11]:
DeltaMtotal=np.sqrt((0.001)**2+(DeltaMale)**2)
print('M_total', round(np.mean(lecturas),3), '±', round(DeltaMtotal,3), 'nC')

M_total 35.717 ± 0.001 nC


También hay **fugas en el electrómetro**, y esto se puede calcular de la siguiente manera:

In [12]:
print('M_fugas=', round(np.mean(fugas),3), '±', 0.001, 'nC')

M_fugas= 0.028 ± 0.001 nC


Ahora ya para calcular el error de la dosis absorbidas, recordamos también los valores del factor de calibración de la cámara en condiciones de referencia ($N_{D,w,Q_0}$), y los factores de corrección ($k_{Q,Q_0}, k_{sat}$). Ya con toda esta información, calculamos el error de la dosis absorbida por las variables con las derivadas parciales de la siguiente expresión:
$$
D_{w,Q}=𝛷_{pT}(M_{total}-M_{fugas})N_{D,w,Q_0}k_{Q,Q_0}k_{sat}k_{pol}
$$

Que se calcula a continuación con el siguiente código:

In [13]:
M_total=np.mean(lecturas)*(10**-9)
DM_lecturas=DeltaMtotal*(10**-9)
M_fugas=fugas[0]*(10**-9)
DM_fugas=0.001*(10**-9)
M_Q=φpT*(M_total-M_fugas)
N_DwQ0=5.396e7
DN_DwQ0=0.001e7
k_QQ0=k_694
Dk_QQ0=error_k_694
k_sat = 1.000
Dk_sat=0.001
k_pol = 1
D_wQ=M_Q*N_DwQ0*k_QQ0*k_sat*k_pol
print(D_wQ)
DeltaD_wQ=np.sqrt((φpT*N_DwQ0*k_QQ0*k_sat*k_pol*DM_lecturas)**2+(φpT*N_DwQ0*k_QQ0*k_sat*k_pol*DM_fugas)**2+(M_Q*k_QQ0*k_sat*k_pol*DN_DwQ0)**2+(M_Q*N_DwQ0*k_sat*k_pol*Dk_QQ0)**2+(M_Q*N_DwQ0*k_QQ0*k_pol*Dk_sat)**2)
print(DeltaD_wQ)

[2.03280252]
[0.00259707]


Entonces, la **dosis absorbida suministrada por el acelerador** es:

In [14]:
print('D_wQ=', round(np.mean(D_wQ[0]),3), '±', round(DeltaD_wQ[0],3), 'Gy')

D_wQ= 2.033 ± 0.003 Gy


Para la **dosis absorbida suministrada por el output**, las unidades nominales no tienen error por construcción, y su incertidumbre entonces será:

In [15]:
D_UM=D_wQ/UM_nominales*100
DeltaD_UM=DeltaD_wQ/UM_nominales*100
print(D_UM)
print(DeltaD_UM)
print('La dosis absorbida suministrada por el output D_UM:', round(np.mean(D_UM[0]),3),'±', round(DeltaD_UM[0],3), 'cGy')

[1.01640126]
[0.00129853]
La dosis absorbida suministrada por el output D_UM: 1.016 ± 0.001 cGy


## Comprobaciones y simulaciones estadísticas

En el Servicio de Radioterapia en el que estamos realizando estas medidas se tiene el criterio de recalibrar el acelerador, ajustar de nuevo las ganancias de las cámaras, si la variación del output respecto al valor nominal de 1 es mayor del 1.5%.

En la teoría del control de calidad se distinguen dos conceptos: *tolerancia* e *incertidumbre*.
- **Tolerancia**: Rango de la variable de control que daría lugar a resultados aceptables. Por ejemplo se consideran tolerables variaciones en la dosis final que reciben los pacientes inferiores al 5% porque no ha sido posible detectar diferencias clínicas en los resultados de tratamientos realizados con esas variaciones.

- **Incertidumbre**: Rango en el que se espera se encuentre el valor *real* de una magnitud medida.

En general se requiere que la variable de control se mida con una incertidumbre inferior a la tolerancia, idealmente incluso despreciable.

El tratamiento combinado de  la tolerancia y la incertidumbre lleva al concepto de *nivel de acción*, como el umbral para el resultado del control que indica la necesidad de realizar alguna corrección.

Dependiendo cuánto de crítico sea el resultado de nuestro control se definen dos posibles formas de situar el nivel de acción:
- **Nivel de acción mayor que la tolerancia**: Se aumenta la tolerancia con la incertidumbre. Expandimos la tolerancia porque aceptamos que los resultados puedan salirse de tolerancia si hay una cierta probabilidad de que estén dentro de tolerancia por su incertidumbre. Por ejemplo si los radares de la DGT miden la velocidad con una incertidumbre del 4% (k=3) solo nos multan si miden que nuestra velocidad supera el límite señalado en más de un 4%. En un tramo a  120 km/h multan si miden más de 124.8 km/h porque de esa manera el 99% de los multados realmente iban a más de 120. Solo hay un 1% de probabilidad de recibir una multa injusta.
- **Nivel de acción inferior la tolerancia**: Se reduce la tolerancia con la incertidumbre. Disminuimos la tolerancia porque no aceptamos los resultados que aunque se encuentren dentro de tolerancia tienen una cierta probabilidad de indicar un resultado fuera de tolerancia si se considera su incertidumbre. Por ejemplo si lo que medimos es la resistencia de una viga no podemos permitir medidas dudosas, aquí lo que hacemos es rechazar *de más*, queremos reducir la probabilidad de *falsos aceptados*.

## Cuestiones
4. **Sabiendo que en el Servicio de Radioterapia considerado se ha tomado el criterio de definir el nivel de acción mayor que la tolerancia responder razonadamente si es necesario ajustar las ganancias o no a la vista de los resultados obtenidos.**

*Nota*: En esta cuestión se valorará positivamente que cada alumno compare sus resultados con otros compañeros de clase (idealmente con todos) y que discutan qué decisión toman sobre las ganancias.

Según lo dicho en el propio enunciado, la variación del output respecto al valor nominal de 1, debería ser menor al $1.5\%$. Esto hace que el rango en el que se mueve su valor será $0.985-1.015$
cGy. Esto significará que el nivel de acción es mayor que la tolerancia, por lo tanto, si el acelerador está dentro de dicho intervalo, estará calibrado. En el caso de caer fuera de él estará descalibrado y habría que recalibrarlo. A continuación calculamos donde cae:


In [16]:
if (D_UM-DeltaD_UM)>=0.985 and (D_UM+DeltaD_UM)<=1.015:
  print('D_UM=', round(np.mean(D_UM[0]),3), 'cGy. No es neccesario ajustar ganancias')
else:
  print('D_UM=', round(np.mean(D_UM[0]),3), '±', round(DeltaD_UM[0],3), 'cGy. Sí es necesario ajustar ganancias')

D_UM= 1.016 ± 0.001 cGy. Sí es necesario ajustar ganancias


Vemos que sale un valor mayor a 1.015, por lo que no está calibrado. A continuación lo comparamos con los resultados de los demás compañeros de clase:

| Nombre                     | Output (cGy/UM) | Incertidumbre (cGy) | Error Relativo | Necesario ajustar ganancia |
|----------------------------|----------------|---------------------|---------------|---------------------------|
| Ignacio                    | 1.0130         | 0.00172841          | 1.299%        | NO                        |
| Darío                      | 1.0130         | 0.011               | 1.300%        | NO                        |
| Helena                     | 1.0130         | 0.0012              | 1.300%        | NO                        |
| Ángela                     | 1.0108         | 0.00129148          | 1.083%        | NO                        |
| Carlota                    | 1.0071         | 0.0013              | 0.710%        | NO                        |
| Sara                       | 1.0118         | 0.00082             | 1.177%        | NO                        |
| Tomás                      | 1.0164         | 0.01249             | 1.640%        | SI                        |
| Umi                        | 1.0168         | 0.01113             | 1.680%        | SI                        |
| Ruth                       | 1.0120         | 0.00143667          | 1.200%        | NO                        |
| Sara FC                    | 1.0147         | 0.00201             | 1.472%        | NO                        |
| Patri                      | 1.0160         | 0.0035              | 1.600%        | SI                        |
| Gonzalo Pascual García     | 1.0093         | 0.0019              | 0.930%        | NO                        |
| Germán                     | 1.0176         | 0.011293            | 1.756%        | SI                        |
| Irene                      | 1.0111         | 0.001058            | 1.110%        | NO                        |
| Samira                     | 1.0112         | 0.0014              | 1.120%        | NO                        |
| Lucía                      | 1.0147         | 0.01122             | 1.470%        | NO                        |
| Sofía                      | 1.0160         | 0.001               | 1.600%        | SI                        |


De la lista de 17 personas, somos 5 los que nos sale fuera de la calibración. Entonces se saca la conclusión de que un $29,4\%$ de los casos en diferentes condiciones ambientales, no cumplen estar dentro de los valores del nivel de acción mayor del acelerador. Por lo tanto, hay una cantidad mayor al $1\%$ de los aceleradores que no entran dentro del nivel de acción mayor, por lo tanto, se debe recalibrar en general. Conseguir el $1\%$ de los aceleradores no estén calibrados como máximo significaría que el $0.17\%$ estuvieran fuera del rango, es decir, que ninguno lo estuviera. Como no es el caso, habría que recalibrar.

5. **Comparar las desviaciones estándar de nuestras medidas con la estimación de incertidumbres realizada en la cuestión 3. Si son diferentes comentar cómo se justifican esas diferencias.**



### Simulaciones

#### Teorema del límite central
Las funciones del módulo `simcaliblinac` están preparadas para cambiar el tamaño de la muestra de medidas y repetir la toma de muestras tantas veces como queramos. El tamaño de la muestra se indica pasando el parámetro **n** y el número de experimentos pasando el parámetro **trials**.

Ejecutando la siguiente celda obtenemos treinta muestras diferentes de cinco medidas cada una cuando fijamos 200 UM y lo almacenamos en el array de numpy muestras. Como resultado de la celda mostramos las dimensiones del array.

In [ ]:
sample_size=5
trial_number=30
muestras = scl.lectura_medida_f(UM=UM_nominales, n=sample_size, trials=trial_number)
muestras.shape

Si queremos mostrar el contenido de muestras ejecutamos esta celda

In [ ]:
muestras

Podemos calcular estádisticas del total de la muestra

In [ ]:
muestras.mean(), muestras.std()

Podemos construir una colección con las medias y desviaciones estándar de cada experimento

In [ ]:
muestras_distribucion_medias = muestras.mean(axis=1)
muestras_distribucion_sigmas = muestras.std(axis=1, ddof=1)

# Mostrar las muestras de ambas ditribuciones
muestras_distribucion_medias, muestras_distribucion_sigmas

## Cuestión
6. Mostrar numéricamente que se verifica el teorema del límite central, es decir, que la media de las muestras de la distribución de medias es la misma que la media de todas las medidas y que dentro de incertidumbres muestrales el valor esperado de la sigma de la distribución de medias es la sigma de la distrubución total dividida por la raíz cuadrada del tamaño de la muestra

## Cuestiones

7. Hacer una gráfica de la razón entre la desviación estándar de la distribución de medias y la sigma de la distrubución total en función del número de experimientos. Discutir si tiene un comportamiento asintótico y en su caso indicar a qué valor tiende.

**Sugerencia**: Juntar el código mostrado en las celdas anteriores para definir una función dependiente del número de experimentos que devuelva la razón entre la media de la distrubución de sigmas y la sigma de la distrubución total. Generar un rango de número de experimentos, pasárselo a la función y hacer la gráfica.

8. Hacer una gráfica de la razón entre la media de la distrubución de sigmas y la sigma de la distrubución total en función del tamaño de la muestra. Analizar la importancia del parametro `ddof` para evitar el sesgo por el tamaño de la muestra. Ver la documentación de la función [numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)

**Sugerencia**: Juntar el código mostrado en las celdas anteriores para definir una función dependiente del tamaño de la muestra que devuelva la razón pedida.

### Cálculo de incertidumbre
Una estimación realista de incertidumbres necesita en general incorporar el conocimiento del experimentador sobre la reproducibilidad de sus aparatos de medida. Son lo que se conoce como **incertidumbres de tipo B** que permiten estimar las sigmas de las distribuciones que siguen los elementos que participan en la medida.

La propagación de estas incertidumbres se puede hacer mediante aproximaciones analíticas, pero en casos complejos estos cálculos se pueden ver limitados por emplear solo aproximaciones de primer orden y por ignorar las covarianzas entre variables.

Una alternativa es emplear técnicas de Monte Carlo. Se pueden muestrear los valores de los elementos que intervienen en la medida si se conocen las distribuciones que siguen y se estiman sus sigmas. Combinando estos valores muestreados mediante la expresión física de la magnitud medida obtenemos una muestra de medidas cuya sigma es la incertidumbre que queremos determinar.

## Cuestión.

9. Hacer una estimación de incertidumbres mediante técnicas de Monte Carlo

**Sugerencia**:  Utilizar las funciones del módulo `simcaliblinac` para hacer experimentos de calibración en los que se obtiene en cada uno de ellos una medida de fugas, una medida de presión y temperatura y la correspondiente muestra de lecturas en el acelerador.